# LLM_1 - **10%** vs. LLM_2 - **10%**

In [1]:
import chess
from stockfish import Stockfish
from ollama import chat
from ollama import ChatResponse
import json
import random
import pandas as pd
import asyncio

In [ ]:
# STOCKFISH_PATH = r"C:\Users\nafis\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"

STOCKFISH_ELO = 3500
NUM_GAMES = 100
MOVE_LIMIT = 300
RANDOM_START = 0
QWEN_US_AI = 10
QWEN_CN_AI = 10

In [3]:
stockfish = Stockfish()
stockfish.update_engine_parameters({"Threads": 4, "Hash": 4096})

In [4]:
qwen_cn_wins = 0
qwen_us_wins = 0
draws = 0

In [5]:
def valid_move_to_string(board):
    legal_moves = [
        board.san(move) for move in board.legal_moves]
    
    if not legal_moves:
        return "No legal moves."
    else:
        return ", ".join(legal_moves)

In [6]:
def get_move_history(board: chess.Board) -> str:
    """Return a formatted move history string like '1. e4 e5 2. Nf3 d5'."""
    temp_board = chess.Board()  # Start from the initial position.
    move_history = []
    
    # Iterate over the moves in the board's move stack.
    for i, move in enumerate(board.move_stack):
        # Get the SAN for the move in the current temporary board.
        move_san = temp_board.san(move)
        # Push the move so that the board state is updated.
        temp_board.push(move)
        
        # For white moves (even index), start a new entry with move number.
        if i % 2 == 0:
            move_history.append(f"{i//2 + 1}. {move_san}")
        # For black moves, append to the last white move entry.
        else:
            move_history[-1] += f" {move_san}"
    
    # Join all entries with a space.
    return " ".join(move_history)

In [7]:
async def get_move_with_timeout(stockfish, timeout=15):
    """
    Run stockfish.get_best_move() in a separate thread and wait for a maximum of `timeout` seconds.
    If the call exceeds the timeout, return None.
    """
    try:
        # Use asyncio.to_thread to run the blocking call in a separate thread.
        move = await asyncio.wait_for(asyncio.to_thread(stockfish.get_best_move), timeout)
        return move
    except asyncio.TimeoutError:
        # move = stockfish.get_best_move_time(1000)
        print(f"Timeout: stockfish.get_best_move() took more than 15 seconds.")
        return None

In [8]:
def get_qwen_move(board:chess.Board, fen:str, temp=0.0):
    qwen_move = None

    valid_moves = valid_move_to_string(board)

    move_history = get_move_history(board)
    
    response: ChatResponse = chat(model='qwen:32b', messages=[
        {
            'role': 'system',
            'content': 'You are a grand master chess player.'
        },
        {
            'role': 'user',
            'content': f"""You are playing chess and it is your turn. This is the current state of the game. Use this to work out where the pieces are on the board:

FEN: {fen}

The possible set of legal moves are: 

Legal Moves: {valid_moves}

+(You have to choose one from the provided list. Do not choose a move that is not in the list.)

The move history is: {move_history}.

Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.
{{
    "san": "The move in SAN format",
}}"""}], options={"temperature": temp})
    
    try:
        parsed_json = json.loads(response.message.content)
        qwen_move = parsed_json["san"]
    except Exception as e:
        print(f"Wrong format given by Qwen. \nQwen Output: {response.message.content}")

    

    return qwen_move

In [9]:
def get_random_move(board:chess.Board, fen:str, seed: int = 101):
    legal_moves = [
        board.san(move)
        for move in board.legal_moves
    ]

    random.seed(seed)

    # print(random.choice(legal_moves))
    return random.choice(legal_moves)

    # print(legal_moves[0])

In [10]:
for game_number in range(1, NUM_GAMES + 1):
    
    move_counter = 0

    print(f"Starting game {game_number}...")

    board = chess.Board()

    while not board.is_game_over() and move_counter <= MOVE_LIMIT:
        if board.turn == chess.WHITE and not board.is_game_over():
            num = random.randint(1,100)
            
            if num <= QWEN_US_AI:
                stockfish.set_fen_position(board.fen())
                qwen_us_move = await get_move_with_timeout(stockfish)

                if qwen_us_move is None:
                    print("No move received. Reinitializing Stockfish.")
                    stockfish = Stockfish(parameters={"Threads": 4, "Hash": 4096})  # Reinitialize Stockfish for the next game.
                    break
            
            else:
                qwen_us_move = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_us_move)
            except Exception as e:
                print(f"Invalid move by Qwen white. Move: {qwen_us_move}")
                qwen_us_move = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_us_move)
                print(f"New Move: {qwen_us_move}")
                # continue
            
            move_counter += 1        
            print(f"Qwen US move: {qwen_us_move} Move number: {move_counter}")

        elif board.turn == chess.BLACK and not board.is_game_over():
            num = random.randint(1, 100)

            if num <= QWEN_CN_AI:
                stockfish.set_fen_position(board.fen())
                qwen_cn_move = await get_move_with_timeout(stockfish)

                if qwen_cn_move is None:
                    print("No move received. Reinitializing Stockfish.")
                    stockfish = Stockfish(parameters={"Threads": 4, "Hash": 4096})  # Reinitialize Stockfish for the next game.
                    break

            else:
                qwen_cn_move = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_cn_move)
            except Exception as e:
                print(f"Invalid move by Qwen black. Move: {qwen_cn_move}")
                qwen_cn_move = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_cn_move)
                print(f"New Move: {qwen_cn_move}")
                # continue
            move_counter += 1
            print(f"Qwen CN move: {qwen_cn_move} Move number: {move_counter}")   

    # Record the result of the game
    result = board.result()
    if result == "1-0":
        qwen_us_wins += 1
    elif result == "0-1":
        qwen_cn_wins += 1
    else:
        draws += 1

    print(f"Game {game_number} result: {result}\n\n")

Starting game 1...
Qwen US move: e2e4 Move number: 1
Qwen CN move: e7e5 Move number: 2
Qwen US move: Nf3 Move number: 3
Qwen CN move: Nc6 Move number: 4
Qwen US move: Ng5 Move number: 5
Qwen CN move: Nh6 Move number: 6
Qwen US move: Nh3 Move number: 7
Qwen CN move: Ng4 Move number: 8
Qwen US move: Nf4 Move number: 9
Qwen CN move: g4f2 Move number: 10
Qwen US move: Nh3 Move number: 11
Qwen CN move: Be7 Move number: 12
Qwen US move: Ng5 Move number: 13
Qwen CN move: Bxg5 Move number: 14
Invalid move by Qwen white. Move: Bg5
New Move: h4
Qwen US move: h4 Move number: 15
Qwen CN move: Nh3 Move number: 16
Qwen US move: h4g5 Move number: 17
Qwen CN move: Nf4 Move number: 18
Qwen US move: Rxh7 Move number: 19
Qwen CN move: Rxh7 Move number: 20
Qwen US move: Qe2 Move number: 21
Qwen CN move: Qxg5 Move number: 22
Qwen US move: Qa6 Move number: 23
Invalid move by Qwen black. Move: Qxa8
New Move: b6
Qwen CN move: b6 Move number: 24
Qwen US move: Qxc8+ Move number: 25
Qwen CN move: a8c8 Move numbe

In [ ]:
results = []

In [ ]:
results.append({
            "Qwen US Wins": qwen_us_wins,
            "Qwen CN Wins": qwen_cn_wins,
            "Draws": draws,
        })

print(results)

## Swapping sides

In [ ]:
stockfish = Stockfish()
stockfish.update_engine_parameters({"Threads": 4, "Hash": 4096})

In [ ]:
for game_number in range(1, NUM_GAMES + 1):
    
    move_counter = 0

    print(f"Starting game {game_number}...")

    board = chess.Board()

    while not board.is_game_over() and move_counter <= MOVE_LIMIT:
        if board.turn == chess.WHITE and not board.is_game_over():
            num = random.randint(1,100)
            
            if num <= QWEN_US_AI:
                stockfish.set_fen_position(board.fen())
                qwen_us_move = await get_move_with_timeout(stockfish)

                if qwen_us_move is None:
                    print("No move received. Reinitializing Stockfish.")
                    stockfish = Stockfish(parameters={"Threads": 4, "Hash": 4096})  # Reinitialize Stockfish for the next game.
                    break
            
            else:
                qwen_us_move = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_us_move)
            except Exception as e:
                print(f"Invalid move by Qwen white. Move: {qwen_us_move}")
                qwen_us_move = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_us_move)
                print(f"New Move: {qwen_us_move}")
                # continue
            
            move_counter += 1        
            print(f"Qwen US move: {qwen_us_move} Move number: {move_counter}")

        elif board.turn == chess.BLACK and not board.is_game_over():
            num = random.randint(1, 100)

            if num <= QWEN_CN_AI:
                stockfish.set_fen_position(board.fen())
                qwen_cn_move = await get_move_with_timeout(stockfish)

                if qwen_cn_move is None:
                    print("No move received. Reinitializing Stockfish.")
                    stockfish = Stockfish(parameters={"Threads": 4, "Hash": 4096})  # Reinitialize Stockfish for the next game.
                    break

            else:
                qwen_cn_move = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_cn_move)
            except Exception as e:
                print(f"Invalid move by Qwen black. Move: {qwen_cn_move}")
                qwen_cn_move = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_cn_move)
                print(f"New Move: {qwen_cn_move}")
                # continue
            move_counter += 1
            print(f"Qwen CN move: {qwen_cn_move} Move number: {move_counter}")   

    # Record the result of the game
    result = board.result()
    if result == "1-0":
        qwen_cn_wins += 1
    elif result == "0-1":
        qwen_us_wins += 1
    else:
        draws += 1

    print(f"Game {game_number} result: {result}\n\n")

In [ ]:
results = []

In [ ]:
results.append({
            "Qwen US Wins": qwen_us_wins,
            "Qwen CN Wins": qwen_cn_wins,
            "Draws": draws,
        })

print(results)

In [ ]:
df = pd.DataFrame(results)
df.to_csv(f"LLM_{QWEN_US_AI}_vs_LLM_{QWEN_CN_AI}.csv", index=False)
print(f"Results saved to LLM_{QWEN_US_AI}_vs_LLM_{QWEN_CN_AI}.csv")